# Deeper CNN

## GoogLeNet / Inception V1

In [1]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import load_model
#https://github.com/jrosebr1/imutils
from imutils import paths
import imutils
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import pickle
from tensorflow import reset_default_graph
import skimage

Using TensorFlow backend.


In [2]:
from keras import layers
from keras.models import Model

from functools import partial

#224x224 image

conv1x1 = partial(layers.Conv2D, kernel_size=1, activation='relu')
conv3x3 = partial(layers.Conv2D, kernel_size=3, padding='same', activation='relu')
conv5x5 = partial(layers.Conv2D, kernel_size=5, padding='same', activation='relu')

def inception_module(in_tensor, c1, c3_1, c3, c5_1, c5, pp):
    conv1 = conv1x1(c1)(in_tensor)

    conv3_1 = conv1x1(c3_1)(in_tensor)
    conv3 = conv3x3(c3)(conv3_1)

    conv5_1 = conv1x1(c5_1)(in_tensor)
    conv5 = conv5x5(c5)(conv5_1)

    pool_conv = conv1x1(pp)(in_tensor)
    pool = layers.MaxPool2D(3, strides=1, padding='same')(pool_conv)

    merged = layers.Concatenate(axis=-1)([conv1, conv3, conv5, pool])
    return merged

def aux_clf(in_tensor, n_classes=1000):
    avg_pool = layers.AvgPool2D(5, 3)(in_tensor)
    conv = conv1x1(128)(avg_pool)
    flattened = layers.Flatten()(conv)
    dense = layers.Dense(1024, activation='relu')(flattened)
    dropout = layers.Dropout(0.7)(dense)
    out = layers.Dense(n_classes, activation='softmax')(dropout)
    return out

def inception_net(in_shape=(224,224,3), n_classes=1000, opt='sgd'):
    in_layer = layers.Input(in_shape)

    conv1 = layers.Conv2D(64, 7, strides=2, activation='relu', padding='same')(in_layer)
    pad1 = layers.ZeroPadding2D()(conv1)
    pool1 = layers.MaxPool2D(3, 2)(pad1)
    conv2_1 = conv1x1(64)(pool1)
    conv2_2 = conv3x3(192)(conv2_1)
    pad2 = layers.ZeroPadding2D()(conv2_2)
    pool2 = layers.MaxPool2D(3, 2)(pad2)

    inception3a = inception_module(pool2, 64, 96, 128, 16, 32, 32)
    inception3b = inception_module(inception3a, 128, 128, 192, 32, 96, 64)
    pad3 = layers.ZeroPadding2D()(inception3b)
    pool3 = layers.MaxPool2D(3, 2)(pad3)

    inception4a = inception_module(pool3, 192, 96, 208, 16, 48, 64)
    inception4b = inception_module(inception4a, 160, 112, 224, 24, 64, 64)
    inception4c = inception_module(inception4b, 128, 128, 256, 24, 64, 64)
    inception4d = inception_module(inception4c, 112, 144, 288, 32, 48, 64)
    inception4e = inception_module(inception4d, 256, 160, 320, 32, 128, 128)
    pad4 = layers.ZeroPadding2D()(inception4e)
    pool4 = layers.MaxPool2D(3, 2)(pad4)

    aux_clf1 = aux_clf(inception4a, n_classes)
    aux_clf2 = aux_clf(inception4d, n_classes)

    inception5a = inception_module(pool4, 256, 160, 320, 32, 128, 128)
    inception5b = inception_module(inception5a, 384, 192, 384, 48, 128, 128)
    pad5 = layers.ZeroPadding2D()(inception5b)
    pool5 = layers.MaxPool2D(3, 2)(pad5)

    avg_pool = layers.GlobalAvgPool2D()(pool5)
    dropout = layers.Dropout(0.4)(avg_pool)
    preds = layers.Dense(n_classes, activation='softmax')(dropout)
    
    
    model = Model(in_layer, [preds, aux_clf1, aux_clf2])
    model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])
    return model

In [42]:
model = inception_net(n_classes=3, opt=sgd4)

In [4]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 114, 114, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
def pre_process(width, height, path):
    """
    Resize and rescale images stored in image folder.
    
    Return pre-processed data and labels for the classes based
    on sub-directories in the image folder
    """
    #containers for pre-processed image data and class labels
    data = []
    labels = []

    #images directory contains 3 sub-directories: 'poison_ivy', 'poison_oak', 'poison_sumac'
    #randomly get image paths and shuffle them
    # current path 'C:\\Users\\jltsa\\Desktop\\Project_2\\images'
    image_paths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(image_paths)

    #preprocess images to width x height pixels as required for LeNet
    for image_path in image_paths:
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is not None:
            shorter_edge = min(image.shape[:2])
            crop_height = (image.shape[0] - shorter_edge) / 2
            crop_width = (image.shape[1] - shorter_edge) / 2
            cropped_img = image[int(crop_height):int(crop_height + shorter_edge),
                              int(crop_width):int(crop_width + shorter_edge)]
            image = skimage.transform.resize(image, (width, height))
            image = img_to_array(image)
            data.append(image)
    
            #Extract class labels
            label = image_path.split(os.path.sep)[-2]
            if label == 'poison_ivy':
                label = 0
            elif label == 'poison_oak':
                label = 1
            else:
                label = 2
            labels.append(label)
       
    #Scal pixel intensities from 0 to 1
    data = np.array(data, dtype='float') / 255.0
    labels = np.array(labels)
    
    return data, labels

In [4]:
#Initialize global training variables

EPOCHS = 15
#Learning rate
#LR = 1e-3
#Batch Size
BS = 10
path_to_img = 'C:\\Users\\jltsa\\Desktop\\Project_2\\images'
path_to_models = 'models\\'
test_path = 'C:\\Users\\jltsa\\Desktop\\Project_2\\Incep_test_img'
train_path ='C:\\Users\\jltsa\\Desktop\\Project_2\\Incep_train_img'

In [5]:
#To increase the amount of training data, build an image generator using data augmentation
aug_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                            height_shift_range=0.1, shear_range=0.2,
                            zoom_range=0.2, horizontal_flip=True, fill_mode='nearest',
                            rescale=1.0/255)

train_gen = aug_gen.flow_from_directory(train_path, target_size=(224,224),
                                       batch_size=BS, color_mode='rgb',
                                       class_mode='categorical', seed=42)
test_gen = aug_gen.flow_from_directory(test_path, target_size=(224,224),
                                       batch_size=BS, color_mode='rgb',
                                       class_mode='categorical', seed=42)
print(train_gen.class_indices)
print(test_gen.class_indices)

Found 579 images belonging to 3 classes.
Found 145 images belonging to 3 classes.
{'poison_ivy': 0, 'poison_oak': 1, 'poison_sumac': 2}
{'poison_ivy': 0, 'poison_oak': 1, 'poison_sumac': 2}


In [6]:
sgd4 = SGD(lr=0.001, momentum=0.9, nesterov=True)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def train_model(im_path, model_path, name):

    data, labels = pre_process(224, 224, im_path)
    #Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
    #One hot encoding
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    

    #create a checkpoint to store the best weights of the model
    #to use these weights later, initialize the same model architecture that the weights were trained from
    #then call model.load_weights('best_weights_alex.h5')
    #can make predictions model.predict_classes(data)
    model_path = model_path+name+".h5"
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
    #add early stopping if 
    earlystop = EarlyStopping(monitor='val_loss', patience=4)
        
    callbacks_list=[checkpoint, earlystop]
        
    #Initialize model
    #model = inception_net(n_classes=3, opt=sgd4)
    #opt_alex = Adam(lr=LR, decay=LR / EPOCHS)


    #fit model using a generator to increase variation and amount of data
    model.fit(X_train, [y_train,y_train,y_train], batch_size=BS,
                    validation_data=(X_test, [y_test,y_test,y_test]),
                    epochs=EPOCHS, callbacks=callbacks_list, verbose=1)
    #save history for loss and val scores
    #loss.append(model.history.history['loss'])
    #val_loss.append(model.history.history['val_loss'])
    #acc.append(model.history.history['acc'])
    #val_acc.append(model.history.history['val_acc'])

In [47]:
train_model(path_to_img, path_to_models, "GoogLeNet")

Train on 543 samples, validate on 181 samples
Epoch 1/15
543/543 [==============================] - 7s 12ms/step - loss: 3.0223 - dense_35_loss: 1.0047 - dense_32_loss: 1.0079 - dense_34_loss: 1.0098 - dense_35_acc: 0.4641 - dense_32_acc: 0.4530 - dense_34_acc: 0.4420 - val_loss: 2.9647 - val_dense_35_loss: 0.9887 - val_dense_32_loss: 0.9883 - val_dense_34_loss: 0.9877 - val_dense_35_acc: 0.4530 - val_dense_32_acc: 0.4530 - val_dense_34_acc: 0.4530
Epoch 2/15
543/543 [==============================] - 5s 9ms/step - loss: 3.0163 - dense_35_loss: 1.0038 - dense_32_loss: 1.0037 - dense_34_loss: 1.0089 - dense_35_acc: 0.4641 - dense_32_acc: 0.4586 - dense_34_acc: 0.4604 - val_loss: 2.9651 - val_dense_35_loss: 0.9885 - val_dense_32_loss: 0.9883 - val_dense_34_loss: 0.9884 - val_dense_35_acc: 0.4530 - val_dense_32_acc: 0.4530 - val_dense_34_acc: 0.4530
Epoch 3/15
543/543 [==============================] - 5s 9ms/step - loss: 3.0227 - dense_35_loss: 1.0032 - dense_32_loss: 1.0090 - dense_34_l

In [12]:
ivy1 = 'C:\\Users\\jltsa\\Desktop\\Project_2\\test_images\\poison_ivy\\ivy1.jpg'

In [13]:
image = cv2.imread(ivy1,cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
orig_im = image.copy()
#pre-process image
image = cv2.resize(image, (224,224))
image = image.astype('float') / 255.0
image = img_to_array(image)
#get image dimensions right to feed into model
image = np.expand_dims(image, axis=0)

In [48]:
model.predict(image)

[array([[0.4633124 , 0.38788423, 0.14880343]], dtype=float32),
 array([[0.4536396 , 0.38770023, 0.15866019]], dtype=float32),
 array([[0.46099177, 0.384503  , 0.15450525]], dtype=float32)]

In [40]:
model.history.history

{'val_loss': [3.2091762347774613,
  3.138313068210749,
  3.0679498219358328,
  3.0216708789214244,
  2.9989414280949376,
  2.9829947829905135,
  2.9751244618747776,
  2.971659540471451,
  2.9703467590374184,
  2.970414967826717,
  2.9745374700641105,
  2.9690299626872028,
  2.9676451406426194,
  2.9649496197041887,
  2.964854023074577],
 'val_dense_30_loss': [1.07136518125376,
  1.0508843864525221,
  1.031899886236665,
  1.016637652289143,
  1.0063089382582606,
  0.9990002970669151,
  0.9946855076110166,
  0.9920631903969781,
  0.9908753116486481,
  0.9901804874615115,
  0.9908432259085429,
  0.9898334273975857,
  0.989597389052586,
  0.9887376582424944,
  0.9887658584842366],
 'val_dense_27_loss': [1.062477394362181,
  1.0321382511386554,
  1.0038091866350964,
  0.9923989535695281,
  0.991276403817024,
  0.9900333117384937,
  0.9896872659414513,
  0.989236838580495,
  0.9892770595313436,
  0.9896890089656767,
  0.9925604943412444,
  0.9896642322039736,
  0.9886292047922124,
  0.987932

In [8]:
def train_model2(model, im_path, model_path, name):
    #create a checkpoint to store the best weights of the model
    #to use these weights later, initialize the same model architecture that the weights were trained from
    #then call model.load_weights('best_weights_alex.h5')
    #can make predictions model.predict_classes(data)
    model_path = model_path+name+".h5"
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
    #add early stopping if 
    earlystop = EarlyStopping(monitor='val_loss', patience=4)
        
    callbacks_list=[checkpoint, earlystop]

    #fit model using a generator to increase variation and amount of data
    model.fit_generator(train_gen, steps_per_epoch=579//BS,
                       epochs=EPOCHS, validation_data=test_gen,
                       callbacks=callbacks_list, validation_steps=145//BS, verbose=1)
    #save history for loss and val scores
    #loss.append(model.history.history['loss'])
    #val_loss.append(model.history.history['val_loss'])
    #acc.append(model.history.history['acc'])
    #val_acc.append(model.history.history['val_acc'])

### Use Keras implenmentation of Inception V3

In [9]:
#load imagenet weights
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
#have 3 classes
predictions = Dense(3, activation='softmax')(x)

model_incepv3 = Model(input=base_model.input, output=predictions)

C:\Users\jltsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [10]:
model_incepv3.compile(optimizer=sgd4, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
train_model2(model_incepv3, path_to_img, path_to_models, "InceptionV3")

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
57/57 [==============================] - 32s 558ms/step - loss: 0.9255 - acc: 0.5509 - val_loss: 0.8699 - val_acc: 0.6214
Epoch 2/15
57/57 [==============================] - 17s 294ms/step - loss: 0.6748 - acc: 0.7084 - val_loss: 0.6501 - val_acc: 0.7481
Epoch 3/15
57/57 [==============================] - 17s 297ms/step - loss: 0.4899 - acc: 0.7873 - val_loss: 0.6206 - val_acc: 0.7111
Epoch 4/15
57/57 [==============================] - 17s 295ms/step - loss: 0.3743 - acc: 0.8559 - val_loss: 0.5731 - val_acc: 0.7630
Epoch 5/15
57/57 [==============================] - 17s 298ms/step - loss: 0.3504 - acc: 0.8663 - val_loss: 0.7194 - val_acc: 0.7259
Epoch 6/15
57/57 [==============================] - 17s 304ms/step - loss: 0.3276 - acc: 0.8715 - val_loss: 0.4232 - val_acc: 0.8370
Epoch 7/15
57/57 [==============================] - 17s 293ms/step - loss: 0.2660 - acc: 0.9018 - val_loss: 0.6198 - val_acc: 0.7407
Epoch 8/15
57/57 [===

In [14]:
model_incepv3.predict(image)

array([[9.9963474e-01, 6.0461203e-05, 3.0485724e-04]], dtype=float32)

In [15]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma